In [1]:
#Libraries used in the functions below
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

from jupyter_dash import JupyterDash
import os
try:
    os.environ.pop('http_proxy')
    os.environ.pop('https_proxy')
except KeyError:
    pass

# Import dash
import dash
from dash import Dash, html, dcc, Input, Output, dash_table, State

# Import plotly
import plotly.graph_objs as go
import plotly.express as px
import seaborn as sns
pd.options.mode.chained_assignment = None 

In [2]:
#Datasets turned into Pandas DataFrames
dfC = pd.read_csv("dataset/courses.csv")
dfSA=pd.read_csv("dataset/studentAssessment.csv")
dfSI=pd.read_csv("dataset/studentInfo.csv")
dfSR=pd.read_csv("dataset/studentRegistration.csv")
dfVLE=pd.read_csv("dataset/vle.csv")
dfSVLE=pd.read_csv("dataset/studentVle.csv")
dfA=pd.read_csv("dataset/assessments.csv")
dfP=pd.read_csv("dataset/generalized_predicts.csv")

In [3]:
dfC['Course_Section'] = dfC[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
course_section=list(dfC['Course_Section'] )
dfP['Course_Section'] = dfP[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)

In [4]:
dfSI['Course_Section'] = dfSI[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
def roster(course_section,sort_type):
    dfRoster=pd.merge(dfSI,dfP, on=['id_student','Course_Section'])
    dfClassRoster=dfRoster[dfRoster['Course_Section']==course_section]
    if sort_type=='Numerical':
        dfClassRoster=dfClassRoster.sort_values('id_student')
    else:
        dfClassRoster=dfClassRoster.sort_values('probability',ascending=False)
    return list(dfClassRoster['id_student'])

In [5]:
dfA['Course_Section'] = dfA[['code_module', 'code_presentation']].apply(lambda x: '-'.join(x), axis=1)
resource_dict = dict(zip(dfVLE['id_site'], dfVLE['activity_type']))
def classCode(Module,Presentation):
    dfCode=dfSVLE[dfSVLE['code_module']==Module]
    dfCode=dfCode[dfCode['code_presentation']==Presentation]
    return dfCode

In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

# Create a unique list of code_module
available_indicators1 = dfC['Course_Section']

# Step 1

app.layout = html.Div([
                        html.Div([
                            html.H1("Instructor Interface/Dashboard"),
                            dcc.Dropdown(
                                id='CourseSection',
                                options=[{'label': i, 'value': i} for i in available_indicators1],
                                value=available_indicators1[0]),
                             dcc.RadioItems(
                                ['Assessment', 'Resources'],
                                'Assessment',
                                id='Graph-Type',
                                inline=True),
                            dcc.Graph(id='GraphMean'),
                            dcc.Graph(id='GraphMedian'),
                                dcc.Slider(
                                    0,
                                    200,
                                    step=None,
                                    id='Day',
                                    value=60),
                        html.Div(id="final_table")],style={'width': '48%','display': 'inline-block'}),
                        html.Div([
                             html.H6("Sort by:"),
                             dcc.RadioItems(
                                ['Numerical', 'Failure Rate'],
                                 id='Sort_by',
                                 value='Numerical',
                            inline=True),
                            
                            dcc.Dropdown(
                                id='StudentID'),
                            
                            html.H6("Table: Selected Student Information"),
                            html.Div(id="Student_table"),
                            html.Div(id="Student_Comments"),

                        ], style={'width': '48%','display': 'inline-block','float': 'right'})
            ])
@app.callback(
    Output('StudentID', 'options'),
    Input('CourseSection', 'value'),Input('Sort_by','value'))
def set_course_options(selected_course,Sort_by):
    return roster(selected_course,Sort_by)

@app.callback(
    Output('StudentID', 'value'),
    Input('StudentID', 'options'))
def set_id_value(available_options):
    return available_options[0]

@app.callback(
    Output('GraphMean', 'figure'),Input('CourseSection', 'value'),Input('StudentID', 'value'),
    Input('Graph-Type','value'),Input('Day','value'))

def update_figure(CourseSection1, StudentID1,type1,day1): 
    if type1=='Assessment':
        dfA1=dfA[dfA['date']<=day1]
        dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
        assessment=list(dfCourse_Assessments['id_assessment'])
        dictMean={}
        for assess in assessment:
            dfSAMean=dfSA[dfSA['id_assessment']==assess]
            dictMean[assess]=np.mean(dfSAMean['score'])
        dfStudentScore=dfSA[dfSA['id_student']==StudentID1]
        dfStudentScore=dfStudentScore[['id_assessment','score']]
        dfStudentScore=dfStudentScore.loc[dfStudentScore['id_assessment'].isin(assessment)]
        dfMean = pd.DataFrame(list(dictMean.items()), columns = ['id_assessment','score'])
        dfMean['type']='Mean'
        dfStudentScore['type']='Student'
        dfTogether=pd.concat([dfStudentScore, dfMean], ignore_index=True,axis=0)
        dfTogether['id_assessment'] = dfTogether.id_assessment.astype(str)
        dfTogether=dfTogether.dropna()
        figure = px.bar(dfTogether, x="id_assessment", y="score", color="type", barmode="group",
                title="Comparison of Students Scores and Class Averages")
        return figure
    else:
        dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
        dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
        dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
        student_count=len(dfCode['id_student'].unique())
        dfCode=dfCode.replace({"id_site": resource_dict})
        dfResource=dfCode.groupby(['id_site']).sum()
        dfResource['sum_click']=dfResource['sum_click']/student_count
        dfResource=dfResource.reset_index()
        dfStudent=dfCode[dfCode['id_student']==StudentID1]
        dfStudent=dfStudent.groupby(['id_site']).sum()
        dfStudent=dfStudent.reset_index()
        dfStudent['type']='Student'
        dfResource['type']='Mean'
        dfTogether1=pd.concat([dfStudent, dfResource], ignore_index=True,axis=0)
        figure2 = px.bar(dfTogether1, x="id_site", y="sum_click", color="type", barmode="group",
            title="Comparison of Students Resource Usage and Class Averages")
        return figure2

@app.callback(
    Output('GraphMedian', 'figure'),Input('CourseSection', 'value'),Input('StudentID', 'value'),
    Input('Graph-Type','value'),Input('Day','value'))

def update_figure(CourseSection1, StudentID1,type1,day1): 
    if type1=='Assessment':
        dfA1=dfA[dfA['date']<=day1]
        dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
        assessment=list(dfCourse_Assessments['id_assessment'])
        dictMedian={}
        for assess in assessment:
            dfSAMedian=dfSA[dfSA['id_assessment']==assess]
            dfSAMedian=dfSAMedian.dropna()
            dictMedian[assess]=np.median(dfSAMedian['score'])
        dfStudentScore=dfSA[dfSA['id_student']==StudentID1]
        dfStudentScore=dfStudentScore[['id_assessment','score']]
        dfStudentScore=dfStudentScore.loc[dfStudentScore['id_assessment'].isin(assessment)]
        dfMedian = pd.DataFrame(list(dictMedian.items()), columns = ['id_assessment','score'])
        dfMedian['type']='Median'
        dfStudentScore['type']='Student'
        dfTogether=pd.concat([dfStudentScore, dfMedian], ignore_index=True,axis=0)
        dfTogether['id_assessment'] = dfTogether.id_assessment.astype(str)
        dfTogether=dfTogether.dropna()
        figure = px.bar(dfTogether, x="id_assessment", y="score", color="type", barmode="group",
                title="Comparison of Students Scores and Class Medians")
        return figure
    else:
        dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
        dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
        dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
        dfCode=dfCode.replace({"id_site": resource_dict})
        dfClass=dfCode.groupby(['id_site','id_student']).sum()
        dfClass=dfClass.reset_index()
        resourceType=dfClass['id_site'].unique()
        resourceMedian={}
        for resource in resourceType:
            dfResourceMedian=dfClass[dfClass['id_site']==resource]
            resourceMedian[resource]=np.median(dfResourceMedian['sum_click'])
        dfMedian = pd.DataFrame(list(resourceMedian.items()), columns = ['id_site','sum_click'])
        dfStudent=dfCode[dfCode['id_student']==StudentID1]
        dfStudent=dfStudent.groupby(['id_site']).sum()
        dfStudent=dfStudent.reset_index()
        dfStudent['type']='Student'
        dfMedian['type']='Median'
        dfTogether1=pd.concat([dfStudent, dfMedian], ignore_index=True,axis=0)
        figure2 = px.bar(dfTogether1, x="id_site", y="sum_click", color="type", barmode="group",
            title="Comparison of Students Resource Usage and Class Medians")
        return figure2

@app.callback(
    Output('final_table', 'children'),Input('CourseSection', 'value'),Input('Graph-Type','value'),Input('Day','value'))

def update_figure(CourseSection1, type1, day1):
    if type1=='Assessment':
        dfA1=dfA[dfA['date']<=day1]
        dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
        assessment=list(dfCourse_Assessments['id_assessment'])
        trial1=[]
        for assess in assessment:
            dfSAScore=dfSA[dfSA['id_assessment']==assess]
            trial1.append([assess]+list(dfSAScore['score'].describe().round(2)))
        data1=pd.DataFrame(trial1,columns=['Assessment','Count','Mean','STD','Min','25%','50%','75%','Max'])
        return [dash_table.DataTable(
                id='Summary',
                columns=[{"name": i, "id": i} for i in data1.columns],
                data=data1.to_dict('records'),
            )]
    else:
        dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
        dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
        dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
        dfCode=dfCode.replace({"id_site": resource_dict})
        dfClass=dfCode.groupby(['id_site','id_student']).sum()
        dfClass=dfClass.reset_index()
        resourceType=dfClass['id_site'].unique()
        trial2=[]
        for idSite in resourceType:
            dfClick=dfClass[dfClass['id_site']==idSite]
            trial2.append([idSite]+list(dfClick['sum_click'].describe().round(2)))
        data2=pd.DataFrame(trial2,columns=['Resource','Count','Mean','STD','Min','25%','50%','75%','Max'])
        return [dash_table.DataTable(
                id='Summary',
                columns=[{"name": i, "id": i} for i in data2.columns],
                data=data2.to_dict('records'),
            )]

@app.callback(
    [Output('Student_table', 'children'),Output('Student_Comments', 'children')],[Input('CourseSection', 'value'),Input('StudentID', 'value'),Input('Day','value')])
def update_figure(CourseSection1, StudentID1, day1):
    dfA1=dfA[dfA['date']<=day1]
    dfCourse_Assessments=dfA1[dfA1['Course_Section']==CourseSection1]
    assessment=list(dfCourse_Assessments['id_assessment'])
    median=[]
    mean=[]
    STDEV=[]
    for assess in assessment:
        dfSAMedian=dfSA[dfSA['id_assessment']==assess]
        mean.append(np.mean(dfSAMedian['score']).round(2))
        STDEV.append(np.std(dfSAMedian['score']).round(2))
        dfSAMedian=dfSAMedian.fillna(0)
        median.append(np.median(dfSAMedian['score']).round(2))
    dfStudentScore=dfSA[dfSA['id_student']==StudentID1]
    dfStudentScore=dfStudentScore[['id_assessment','score']]
    dfStudentScore=dfStudentScore.loc[dfStudentScore['id_assessment'].isin(assessment)]
    for assess in assessment:
        if dfStudentScore[dfStudentScore['id_assessment'].isin([assess])].empty==True:
            dfStudentScore.loc[len(dfStudentScore.index)] = [assess, 0] 
    dfStudentScore['STDEV']=STDEV
    dfStudentScore['Mean']=mean
    dfStudentScore['Median']=median
    dfStudentScore['DifferenceMean']=(dfStudentScore['score']-dfStudentScore['Mean']).round(2)
    dfStudentScore['DifferenceMedian']=(dfStudentScore['score']-dfStudentScore['Median']).round(2)
    dfStudentScore=dfStudentScore[['id_assessment','score','DifferenceMedian','DifferenceMean','STDEV']]
    dfStudentScore.columns=['ID','Student','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)','STDEV']
    dfStudentScore=dfStudentScore.fillna(0)

    dfSVLE1=dfSVLE[dfSVLE['date']<=day1]
    dfCode=dfSVLE1[dfSVLE1['code_module']==CourseSection1[:3]]
    dfCode=dfCode[dfCode['code_presentation']==CourseSection1[4:]]
    dfCode=dfCode.replace({"id_site": resource_dict})
    dfClass=dfCode.groupby(['id_site','id_student']).sum()
    dfClass=dfClass.reset_index()
    resourceType=dfClass['id_site'].unique()
    resourceMedian={}
    resourceMean=[]
    resourceSTDEV=[]
    for resource in resourceType:
        dfResourceMedian=dfClass[dfClass['id_site']==resource]
        resourceMedian[resource]=(np.median(dfResourceMedian['sum_click']).round(2))
        resourceMean.append(np.mean(dfResourceMedian['sum_click']).round(2))
        resourceSTDEV.append(np.std(dfResourceMedian['sum_click']).round(2))
    dfStudent=dfCode[dfCode['id_student']==StudentID1]
    dfStudent=dfStudent.groupby(['id_site']).sum()
    dfStudent=dfStudent.reset_index()
    ResourceList=[resourceMedian.keys(),resourceMedian.values()]
    dfResourceList=pd.DataFrame(ResourceList).T
    dfResourceList['Mean']=resourceMean
    dfResourceList['STDEV']=resourceSTDEV
    dfResourceList.columns=['id_site','Median','Mean','STDEV']
    dfStudentResource=pd.merge(dfResourceList, dfStudent, on='id_site',how='left')
    dfStudentResource=dfStudentResource.fillna(0)
    del dfStudentResource['id_student']
    del dfStudentResource['date']
    dfStudentResource['Median']=(dfStudentResource['sum_click']-dfStudentResource['Median']).round(2)
    dfStudentResource['Mean']=(dfStudentResource['sum_click']-dfStudentResource['Mean']).round(2)
    dfStudentResource=dfStudentResource[['id_site','sum_click','Median','Mean','STDEV']]
    dfStudentResource.columns=['ID','Student','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)','STDEV']
    dfTogetherStudent=pd.concat([dfStudentScore, dfStudentResource], ignore_index=True,axis=0)
    count=0
    comment=[]
    text=""
    for thing in dfTogetherStudent['STDEV']:
        if dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0:
            if dfTogetherStudent['Difference (Sum_Click - Mean)'][count]<=0.5*thing:
                comment.append('Average')
            elif dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>=2*thing:
                comment.append('Well Above Average')
            elif dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0.5*thing:
                comment.append('Above Average')
            count=count+1
        else:
            if -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]<=0.5*thing:
                comment.append('Average')
            elif -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>=2*thing:
                comment.append('Well Below Average')
                if type(dfTogetherStudent['ID'][count])==type(14):
                    text=text+"This student performed Well Below Average on the "+str(dfTogetherStudent['ID'][count])+" assessment.\n"
                    text=text+"It is recommended that the student review the content related to the "+str(dfTogetherStudent['ID'][count])+" assessment, perform extra practice, and make appropriate corrections to their assessment.\n\n"
                else:
                    text=text+"This student's level of access to the "+str(dfTogetherStudent['ID'][count])+" resource was Well Below Average.\n"
                    text=text+"If the student is struggling with the course content, it is recommended that they spend more time accessing "+str(dfTogetherStudent['ID'][count])+".\n\n"

            elif -1*dfTogetherStudent['Difference (Sum_Click - Mean)'][count]>0.5*thing:
                comment.append('Below Average')
                if type(dfTogetherStudent['ID'][count])==type(14):
                    text=text+"This student performed Below Average on the "+str(dfTogetherStudent['ID'][count])+" assessment.\n"
                    text=text+"It is recommended that the student review the content related to the "+str(dfTogetherStudent['ID'][count])+" assessment, perform extra practice, and make appropriate corrections to their assessment.\n\n"
                else:
                    text=text+"This student's level of access to the "+str(dfTogetherStudent['ID'][count])+" resource was Below Average.\n"
                    text=text+"If the student is struggling with the course content, it is recommended that they spend more time accessing "+str(dfTogetherStudent['ID'][count])+".\n\n"            
            count=count+1
    dfTogetherStudent['STDEV']=comment
    dfTogetherStudent.columns=['Assessment/Resource','Student Score/Clicks','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)','Performance']
    dfTogetherStudent=dfTogetherStudent[['Assessment/Resource','Student Score/Clicks','Difference (Sum_Click - Median)','Difference (Sum_Click - Mean)']]

    return [dash_table.DataTable(
        id='Summary1',
        columns=[{"name": i, "id": i} for i in dfTogetherStudent.columns],
        data=dfTogetherStudent.to_dict('records'),
        )],[dcc.Textarea(
        id='textarea-example',
        value=text,
        style={'width': '100%', 'height': 300},
        )]


    
    
app.run_server(mode="external", port = 8301,debug=False)

 * Running on http://127.0.0.1:8301 (Press CTRL+C to quit)
127.0.0.1 - - [17/Aug/2022 12:32:51] "GET /_alive_fc2108b4-858c-4f6b-9180-ad7ac7cafa0a HTTP/1.1" 200 -


Dash app running on http://127.0.0.1:8301/


127.0.0.1 - - [17/Aug/2022 12:32:52] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 12:32:53] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_6_0m1659035085.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 12:32:53] "GET /_dash-component-suites/dash/deps/react@16.v2_6_0m1659035085.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 12:32:53] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_6_0m1659035085.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 12:32:53] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_6_0m1659035085.8.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 12:32:53] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_6_0m1659035085.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 12:32:53] "GET /_dash-component-suites/dash/html/dash_html_components.v2_0_4m1659035085.min.js HTTP/1.1" 200 -
127.0.0.1 - - [17/Aug/2022 12:32:53] "GET /_dash-component-suites/dash/dcc/dash_core_components.v2_6_0m1659035085.js HTTP/